1. Installation de la bibliothèque MySQL Connector

In [ ]:
!pip install mysql-connector-python

2. Connexion et extraction de la base de données

In [9]:

import mysql.connector
password = "root"
# Connexion à la base de données
conn = mysql.connector.connect(
    host="localhost",  # Utilise l'adresse IP publique ou localhost si tu fais un tunnel SSH
    user="root",               # Mon nom d'utilisateur MySQL
    password=password,        # Mon mot de passe MySQL
    database="Ecole"            # Le nom de ma base de données
)

# Création d'un curseur pour exécuter les requêtes
cursor = conn.cursor()

# Exécution d'une requête SQL pour extraire les données de la table "eleves"
cursor.execute("SELECT * FROM eleves")

# Récupération des résultats
rows = cursor.fetchall()

# Affichage des données
for row in rows:
    print(row)

# Fermeture du curseur et de la connexion
cursor.close()
conn.close()

(1, 'Mark', 'Watney', 101, '777-555-1234', None, 2035, 5)


3. Manipulation des données

In [11]:
# Connexion à la base de données
conn = mysql.connector.connect(
    host="localhost",  # ou l'adresse IP de ton serveur MySQL
    user="root",       # Nom d'utilisateur MySQL
    password="root",  # Mot de passe MySQL
    database="Ecole"   # Nom de la base de données
)

# Création d'un curseur pour exécuter les requêtes
cursor = conn.cursor()

# 1. Récupérer les données des élèves
cursor.execute("SELECT prenom, nom, numero_classe FROM eleves")
eleves = cursor.fetchall()

# 2. Récupérer les données des enseignants
cursor.execute("SELECT prenom, nom, numero_classe FROM enseignants")
enseignants = cursor.fetchall()

# Fermeture du curseur et de la connexion
cursor.close()
conn.close()

# 3. Associer chaque élève à un enseignant par numéro de classe
for eleve in eleves:
    prenom_eleve, nom_eleve, classe_eleve = eleve
    enseignant_trouve = None

    for enseignant in enseignants:
        prenom_enseignant, nom_enseignant, classe_enseignant = enseignant
        if classe_eleve == classe_enseignant:
            enseignant_trouve = f"{prenom_enseignant} {nom_enseignant}"
            break

    # Afficher l'association élève-enseignant
    if enseignant_trouve:
        print(f"Élève: {prenom_eleve} {nom_eleve}, Classe: {classe_eleve} -> Enseignant responsable: {enseignant_trouve}")
    else:
        print(f"Élève: {prenom_eleve} {nom_eleve}, Classe: {classe_eleve} -> Enseignant responsable: Non trouvé")

Élève: Mark Watney, Classe: 5 -> Enseignant responsable: Jonas Salk


4. Analyse des données

In [14]:
import mysql.connector

# Connexion à la base de données
conn = mysql.connector.connect(
    host="localhost",  # ou l'adresse IP de ton serveur MySQL
    user="root",       # Nom d'utilisateur MySQL
    password="root",  # Mot de passe MySQL
    database="Ecole"   # Nom de la base de données
)

# Création d'un curseur pour exécuter les requêtes
cursor = conn.cursor()

# 1. Récupérer les données des élèves
cursor.execute("SELECT prenom, nom, numero_classe FROM eleves")
eleves = cursor.fetchall()

# 2. Récupérer les données des enseignants
cursor.execute("SELECT prenom, nom, numero_classe FROM enseignants")
enseignants = cursor.fetchall()

# Fermeture du curseur et de la connexion
cursor.close()
conn.close()

# 3. Utiliser un dictionnaire pour compter les élèves par enseignant
enseignant_eleves_count = {}

# Initialiser le dictionnaire avec chaque enseignant
for enseignant in enseignants:
    prenom_enseignant, nom_enseignant, classe_enseignant = enseignant
    enseignant_eleves_count[f"{prenom_enseignant} {nom_enseignant}"] = 0

# Compter le nombre d'élèves par enseignant en fonction de la classe
for eleve in eleves:
    prenom_eleve, nom_eleve, classe_eleve = eleve
    
    # Chercher l'enseignant correspondant à la classe de l'élève
    for enseignant in enseignants:
        prenom_enseignant, nom_enseignant, classe_enseignant = enseignant
        if classe_eleve == classe_enseignant:
            enseignant_eleves_count[f"{prenom_enseignant} {nom_enseignant}"] += 1
            break

# 4. Afficher le nombre d'élèves pour chaque enseignant
for enseignant, count in enseignant_eleves_count.items():
    print(f"{enseignant} a {count} élève(s).")


Jonas Salk a 1 élève(s).


5. Automatisation du pipeline

In [16]:
import mysql.connector

def connect_to_db():
    # Connexion à la base de données
    conn = mysql.connector.connect(
        host="localhost",  # ou l'adresse IP de ton serveur MySQL
        user="root",       # Nom d'utilisateur MySQL
        password="root",  # Mot de passe MySQL
        database="Ecole"   # Nom de la base de données
    )
    return conn

def fetch_data(conn):
    # Création d'un curseur pour exécuter les requêtes
    cursor = conn.cursor()

    # Récupérer les données des élèves
    cursor.execute("SELECT prenom, nom, numero_classe FROM eleves")
    eleves = cursor.fetchall()

    # Récupérer les données des enseignants
    cursor.execute("SELECT prenom, nom, numero_classe FROM enseignants")
    enseignants = cursor.fetchall()

    # Fermer le curseur
    cursor.close()

    return eleves, enseignants

def associate_students_with_teachers(eleves, enseignants):
    # Associer chaque élève à son enseignant par numéro de classe
    associations = []
    for eleve in eleves:
        prenom_eleve, nom_eleve, classe_eleve = eleve
        enseignant_trouve = None
        for enseignant in enseignants:
            prenom_enseignant, nom_enseignant, classe_enseignant = enseignant
            if classe_eleve == classe_enseignant:
                enseignant_trouve = f"{prenom_enseignant} {nom_enseignant}"
                break
        associations.append((f"{prenom_eleve} {nom_eleve}", enseignant_trouve))
    return associations

def count_students_per_teacher(eleves, enseignants):
    # Compter le nombre d'élèves par enseignant en fonction du numéro de classe
    enseignant_eleves_count = {f"{prenom} {nom}": 0 for prenom, nom, _ in enseignants}

    for eleve in eleves:
        classe_eleve = eleve[2]
        for enseignant in enseignants:
            if classe_eleve == enseignant[2]:
                enseignant_nom_complet = f"{enseignant[0]} {enseignant[1]}"
                enseignant_eleves_count[enseignant_nom_complet] += 1
                break

    return enseignant_eleves_count

def display_associations(associations):
    print("\nAssociations élèves-enseignants :")
    for eleve, enseignant in associations:
        if enseignant:
            print(f"{eleve} -> Enseignant responsable: {enseignant}")
        else:
            print(f"{eleve} -> Enseignant responsable: Non trouvé")

def display_student_counts(enseignant_eleves_count):
    print("\nNombre d'élèves par enseignant :")
    for enseignant, count in enseignant_eleves_count.items():
        print(f"{enseignant} a {count} élève(s).")

def main():
    # Connexion à la base de données
    conn = connect_to_db()

    try:
        # Extraction des données
        eleves, enseignants = fetch_data(conn)

        # Association des élèves aux enseignants
        associations = associate_students_with_teachers(eleves, enseignants)

        # Comptage des élèves par enseignant
        enseignant_eleves_count = count_students_per_teacher(eleves, enseignants)

        # Affichage des résultats
        display_associations(associations)
        display_student_counts(enseignant_eleves_count)

    finally:
        # Fermeture de la connexion à la base de données
        conn.close()

# Exécution du script
if __name__ == "__main__":
    main()



Associations élèves-enseignants :
Mark Watney -> Enseignant responsable: Jonas Salk

Nombre d'élèves par enseignant :
Jonas Salk a 1 élève(s).


EXERCICES PIPELINES